In [8]:
from azure.storage.blob import BlobServiceClient

import pandas as pd
from datetime import date
import openpyxl as ex
import re as re
import requests

LOCALFILENAME= 'temp.xlsx'
CONTAINERNAME= 'upload-manuell'
BLOBNAME= 'ga_properties_urls.xlsx'
STORAGEACCOUNTURL= "https://azapexceldev.blob.core.windows.net"
STORAGEACCOUNTKEY= "RqKHccDs4IL1/IZiPzMcIg9XRImdItrdKJEU9bG0nbl7Ckf5pcj86y9h7fDtCS76x136sgBdn/+VhFw8z5BdMg=="

#download from blob

blobClient = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
blobItem = blobClient.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
with open(LOCALFILENAME, "wb") as my_blob:
    blob_data = blobItem.download_blob()
    blob_data.readinto(my_blob)

pd.set_option("display.max_rows", None, "display.max_columns", None)

## Laden Verzeichnis der GA-Views und Properties mit den dazugehörigen URLs


In [9]:
df_properties= pd.read_excel(LOCALFILENAME, engine='openpyxl', sheet_name=None)

In [10]:
df_properties = df_properties['Tabelle1']

In [11]:
df_properties.info()

In [12]:
columns_df_properties = ["account_name", "account_id", "property_name", "property_id", "property_url", "view_name", "view_id", "seitenaufrufe_letzte_360_tage", 'bemerkung']

In [13]:
df_properties.columns = columns_df_properties

## Erstellen Liste der verfügbaren URLs aus GA

In [26]:
url_list = list(df_properties['property_url'].unique())

## Erstellen leeres DF für die Daten aus dem GTM-Crawl

In [16]:
columns = ['request_url', "response_url", 'status', 'gtm', "ua", 'error', 'domain_verkauf']

In [17]:
df_gtm = pd.DataFrame(columns = columns)

## Erstellen Funktionen für den Crawl

In [18]:
def remove_newline(a):
    return re.sub(r'[\n]', '', a)

In [19]:
def extract_gtm(r):
    if re.search(r'GTM-[A-Z0-9]{6,7}', r) is not None:
        return list(set(re.findall(r'GTM-[A-Z0-9]{6,7}', r)))
    else:
        return 'NA'

In [20]:
def extract_ua(r):
    if re.search(r'UA-[0-9-]{9,11}', r) is not None:
        return list(set(re.findall(r'UA-[0-9-]{9,11}', r)))
    else:
        return 'NA'

In [21]:
def domain_verkauf(r):
    if re.search(r'(This domain may be for sale|Diese Website steht zum Verkauf)', r) is not None:
        return "ja"
    else:
        return 'NA'

In [22]:
def fill_row(url,response, response_text):
    row['request_url'] = url
    row['response_url'] = response.url
    row['status'] = response.status_code
    row['error'] = 'NA'
    row['gtm'] = extract_gtm(response_text)
    row['ua'] = extract_ua(response_text)
    row['domain_verkauf'] = domain_verkauf(response_text)

In [23]:
def fill_error(url, e):
    row['request_url'] = url
    row['response_url'] = 'NA'
    row['status'] = 'NA'
    row['gtm'] = 'NA'
    row['ua'] = 'NA'
    row['domain_verkauf'] = 'NA'
    row['error'] = e

## Crawl ausführen, Ergebnisse ins DF abspeichern

In [30]:
for url in url_list:
    length = len(url_list)
    row = {}
    headers = {'User-Agent': 'Safari/605.1.15' }
    print('processing url:'.format(url))
    try:
        response = requests.get(url, headers = headers, timeout = 3)
        response_text = remove_newline(response.text)

        print('response received')
        
        if re.search(r'\t<meta.+<body></body>', response_text) is not None: 
            try:
                redir_url = re.search(r'url=(http.*rhd)',response_text).group(1)
                response_redir = requests.get(redir_url, headers = headers)
                response_redir_text = remove_newline(response_redir.text)
                fill_row(url,response_redir, response_redir_text)
            except Exception as e:
                fill_error(url,e)
                
        else:
            fill_row(url,response, response_text)
            
            
            
    except Exception as e:
        fill_error(url,e)  
    
    df_gtm = df_gtm.append(row, ignore_index = True)
   # print(f'processing item {df_list.index(url)+1} of {length}', end='\r')

In [31]:
df_gtm.info()

In [35]:
df_gtm['property_url'] = df_gtm['request_url']

## Mergen der GA-Daten mit den Daten aus dem Crawl

In [36]:
df_merged = pd.merge(df_properties, df_gtm, on = 'property_url', how = 'left')

In [37]:
df_merged.info()

## Speichern der gemergten Daten ins Blob als CSV

In [39]:
output = df_merged.to_csv(index=False, encoding = "utf-8")

In [40]:
todays_date = date.today()
file_name = "verzeichnis_gtm_container"+"_"+str(todays_date)+".csv"

In [41]:

blobClient = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
container_client = blobClient.get_container_client('upload-manuell')

blob_client = container_client.get_blob_client(file_name)
blob_client.upload_blob(output, blob_type="BlockBlob")
